In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# URLs for datasets
datasets = {
    "SO2TONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/SO2TONS_dataset.csv",
    "NOXTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/NOXTONS_dataset.csv",
    "COTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/COTONS_dataset.csv"
}

# Define the peak season months (May through August)
peak_season_months = [5, 6, 7, 8]

# Define lakes (sources)
sources = ["LAKE-1", "LAKE-2", "LAKE-3", "LAKE-4"]

# Define the specific day for prediction
specific_date = pd.Timestamp("2022-07-15")

# Initialize a dictionary to store models, predictions, and inputs for verification
models = {}
predictions = {}

# Loop through each dataset (SO2TONS, NOXTONS, COTONS)
for parameter, url in datasets.items():
    # Load the dataset
    data = pd.read_csv(url)

    # Convert the 'date' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter for peak season
    data = data[data['date'].dt.month.isin(peak_season_months)]

    # Separate data by source
    for source in sources:
        source_data = data[data['Source'] == source]

        # Check if the source data has enough rows
        if source_data.empty or len(source_data) < 10:
            print(f"Not enough data for {parameter} at {source}. Skipping...")
            continue

        # Define predictors (e.g., weather features)
        predictors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
        target = 'Emissions_Load'

        # Drop rows with missing values
        source_data = source_data.dropna(subset=predictors + [target])

        # Split the data into features (X) and target (y)
        X = source_data[predictors]
        y = source_data[target]

        # Discretize target variable into bins
        n_bins = 5  # You can adjust this value to increase or decrease the number of classes
        discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
        y_binned = discretizer.fit_transform(y.values.reshape(-1, 1)).astype(int).flatten()
        bin_edges = discretizer.bin_edges_[0]

        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binned, test_size=0.2, random_state=42)

        # Define a Logistic Regression model for classification
        model = LogisticRegression(solver='lbfgs')

        # Train the model
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model for {parameter} at {source}:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(classification_report(y_test, y_pred))

        # Save the model and scaler
        models[(parameter, source)] = (model, scaler, discretizer)

        # Check if the specific date exists in the source data
        day_data = source_data[source_data['date'] == specific_date]
        if not day_data.empty:
            # Extract feature values for the specific day
            specific_features = scaler.transform(day_data[predictors])
            specific_actual = discretizer.transform(day_data[[target]].values).astype(int).flatten()[0]
            actual_value_range = (bin_edges[specific_actual], bin_edges[specific_actual + 1])

            # Predict emissions/load category for the specific day
            specific_prediction = model.predict(specific_features)[0]
            predicted_value_range = (bin_edges[specific_prediction], bin_edges[specific_prediction + 1])

            # Save the prediction and actual value for verification
            predictions[(parameter, source)] = {
                "features": day_data[predictors].iloc[0],
                "actual_class": specific_actual,
                "actual_value_range": actual_value_range,
                "predicted_class": specific_prediction,
                "predicted_value_range": predicted_value_range
            }

# Display all predictions at the end
print("\nFinal Predictions:")
for key, value in predictions.items():
    parameter, source = key
    print(f"{parameter} at {source}:")
    print(f"  Features: {value['features'].to_dict()}")
    print(f"  Actual Emissions_Load Class: {value['actual_class']}, Value Range: {value['actual_value_range']}")
    print(f"  Predicted Emissions_Load Class: {value['predicted_class']}, Value Range: {value['predicted_value_range']}")
    print()


Model for SO2TONS at LAKE-1:
  Accuracy: 0.2333
              precision    recall  f1-score   support

           0       0.14      0.20      0.17         5
           1       0.00      0.00      0.00         8
           2       0.40      0.29      0.33         7
           3       0.12      0.25      0.17         4
           4       0.50      0.50      0.50         6

    accuracy                           0.23        30
   macro avg       0.23      0.25      0.23        30
weighted avg       0.23      0.23      0.23        30

Model for SO2TONS at LAKE-2:
  Accuracy: 0.2979
              precision    recall  f1-score   support

           0       0.33      0.33      0.33        12
           1       0.43      0.33      0.38         9
           2       0.50      0.18      0.27        11
           3       0.20      0.60      0.30         5
           4       0.22      0.20      0.21        10

    accuracy                           0.30        47
   macro avg       0.34      0.33  